In [ ]:
!pip install PyPDF2

## Librerias

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

## Cargar datos desde PDF

In [14]:
import PyPDF2
import os

def cargar_datos_desde_pdf(ruta_pdf):
    """
    Cargar datos desde un archivo PDF y extraer su texto.
    :param ruta_pdf: Ruta del archivo PDF
    :return: Texto extraído del PDF
    """
    texto_completo = ""
    try:
        with open(ruta_pdf, 'rb') as archivo:
            lector_pdf = PyPDF2.PdfReader(archivo)
            print(f"El PDF tiene {len(lector_pdf.pages)} página(s).")  # Verificar el número de páginas
            for i, pagina in enumerate(lector_pdf.pages):
                texto_pagina = pagina.extract_text()
                if texto_pagina:  # Verificar si hay texto
                    texto_completo += texto_pagina
                else:
                    print(f"Advertencia: La página {i+1} no contiene texto seleccionable.")
        return texto_completo
    except FileNotFoundError:
        print("Error: El archivo PDF no se encuentra. Verifica la ruta.")
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
    return None

# Verificar si el archivo existe
ruta_pdf = './Futbol.pdf'  # Utiliza la barra inclinada hacia adelante para mayor compatibilidad

if os.path.isfile(ruta_pdf):
    contenido_pdf = cargar_datos_desde_pdf(ruta_pdf)
else:
    print("Error: No se encuentra el archivo PDF.")


El PDF tiene 5 página(s).


## Seleccionamos el Modelo que vamos a utilizar

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Accedemos a la base de datos

In [19]:
vector_store = Chroma(
    collection_name="rag2",
    embedding_function=embeddings,
    persist_directory="./datasets",
)

In [20]:
# Se ve como funciona la base de datos
retriever = vector_store.as_retriever()

## RAG Chain

In [34]:
# Plantilla de conversación
conversation_template = """Please answer the following question based only on the given context:
{context}

Question: {question}
"""

# Crear el prompt a partir de la plantilla
prompt_template = ChatPromptTemplate.from_template(conversation_template)

# LLM local
local_llm = "tinyllama"
local_model = ChatOllama(model=local_llm)

# Cadena de procesamiento
pregunta = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | local_model
    | StrOutputParser()
)